# 🎓 自分だけのGPT-OSS 20Bを作ろう！

## このノートブックでできること

あなたのChatGPTとの会話履歴を使って、**自分専用のAIモデル**を作ることができます。
Google のアカウントを持っていれば、誰でも無料で実行可能です。
最初にこのファイルを、左上の「ファイル」→「ドライブにコピーを保存」して自分のアカウントに保存してください。

その後、以下のような流れで学習を実行することができます!

```mermaid
flowchart LR
    A["📥 あなたの<br>ChatGPT履歴"] --> B["🎓 学習<br>(約10分)"]
    B --> C["🤖 自分専用の<br>GPT-OSS 20B"]
    
    style A fill:#e1f5fe,stroke:#01579b
    style B fill:#fff3e0,stroke:#e65100
    style C fill:#e8f5e9,stroke:#2e7d32
```

### なぜ自分専用モデルを作るの？

- 🎯 あなたの口調・文体を学習
- 📚 あなたがよく話す話題に詳しくなる
- 🔒 ローカルで動かせる（API料金不要）

---

### 🏃 始め方

画面上部の「**ランタイム**」→「**すべてのセルを実行**」をクリックするだけ！

<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>
</div>

---

## 📜 ライセンスと謝辞

このノートブックは [Unsloth](https://github.com/unslothai/unsloth) の
[LGPL-3.0ライセンス](https://github.com/unslothai/notebooks?tab=LGPL-3.0-1-ov-file#readme) のノートブックを元に作成しました。

### 変更点
- 🇯🇵 日本語化
- 💬 ChatGPT会話履歴からのデータ読み込み機能を追加
- 📖 初心者向け解説とビジュアルガイドを追加
- 🔧 データ準備セクションの拡充

元のノートブック: https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/gpt-oss-(20B)-Fine-tuning.ipynb
ライセンス: LGPL-3.0

---

## 📖 知っておくと便利な用語

<details>
<summary>🔧 <b>Fine-tuning（ファインチューニング）</b></summary>

すでに学習済みのAIモデルに、追加で自分のデータを学習させること。  
ゼロから作るより圧倒的に少ないデータ・時間で済みます。

例えるなら：英語が話せる人に日本語を教えるようなもの  
（赤ちゃんにゼロから言語を教えるより早い！）
</details>

<details>
<summary>🧩 <b>LoRA（ローラ）</b></summary>

モデル全体ではなく、一部だけを効率的に学習する技術。  
メモリが少なくても学習できる魔法のような仕組み。

- 通常: 200億パラメータ全部を更新 → メモリ不足
- LoRA: 400万パラメータだけ更新 → OK！
</details>

<details>
<summary>📦 <b>4bit量子化（りょうしか）</b></summary>

モデルのサイズを圧縮する技術。  
画質を落として画像ファイルを小さくするのと似ています。

- 通常: 40GB必要 → 無料GPUでは動かない
- 4bit: 10GB程度 → 無料のT4 GPUで動く！
</details>

<details>
<summary>🤖 <b>GPT-OSS 20B</b></summary>

OpenAIが公開した200億パラメータのオープンソースモデル。  
ChatGPTと似た形式で推論（reasoning）機能を持っています。
</details>

---

## ⚙️ 事前準備（必ず行ってください）

### GPU（グラフィックボード）を有効にする

コードを高速に実行するために、GPUを使います。  
**無料で使えるので安心してください！**

**手順：**
1. 画面上部の「**ランタイム**」をクリック
2. 「**ランタイムのタイプを変更**」を選択
3. 「**ハードウェア アクセラレータ**」で「**T4 GPU**」を選択
4. 「**保存**」をクリック

> ⚠️ **この設定をしないとエラーになります！**

In [ ]:
# @title 📋 準備 & データ設定（このセルで設定を確定）
# @markdown ---
# @markdown ### 1. 環境チェック
google_account = True  # @param {type:"boolean"}
gpu_enabled = False    # @param {type:"boolean"}

# @markdown ---
# @markdown ### 2. データソースを選択
data_source = "sample"  # @param ["sample", "chatgpt"]

# @markdown ---
# @markdown ### 3. ChatGPTエクスポートを使う場合はここでアップロード
upload_now = False  # @param {type:"boolean"}

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 処理
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# 選択肢の説明
OPTIONS = {
    "sample": "サンプルデータ（デモ用・HuggingFace）",
    "chatgpt": "ChatGPTエクスポート"
}
use_chatgpt_export = (data_source == "chatgpt")

print("=" * 50)
print("📋 設定内容")
print("=" * 50)

# 環境チェック
missing = []
if not google_account: missing.append("Googleアカウント")
if not gpu_enabled: missing.append("GPU設定")

if missing:
    print(f"⚠️ 環境: {', '.join(missing)} が未確認")
else:
    print("✅ 環境: OK")

# データソース
print(f"📂 データ: {OPTIONS[data_source]}")

# ChatGPTエクスポートのアップロード
if use_chatgpt_export and upload_now:
    print("\n📤 ファイルアップロード:")
    from google.colab import files
    uploaded = files.upload()
    if 'conversations.json' in uploaded:
        print("✅ conversations.json がアップロードされました！")
    else:
        print("⚠️ conversations.json をアップロードしてください")
elif use_chatgpt_export:
    print("\n💡 ヒント: 「upload_now」にチェックを入れて再実行するとアップロードできます")

print("\n" + "=" * 50)
print("✅ 設定完了！次のセルに進んでください")
print("=" * 50)

---

## 📍 ステップ 1/7 - 環境セットアップ

| ステップ | 状態 |
|---------|------|
| **1. 環境セットアップ** | 🔄 **いまここ** |
| 2. モデル読み込み | ⏳ まだ |
| 3. ChatGPTデータ取得 | ⏳ まだ |
| 4. データ準備 | ⏳ まだ |
| 5. 学習実行 | ⏳ まだ |
| 6. テスト | ⏳ まだ |
| 7. 保存 | ⏳ まだ |

必要なライブラリをインストールします。  
**数分かかりますが、自動で進むので待っていてください。**

In [ ]:
%%capture
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📦 ライブラリのインストール
# このセルは自動で実行されます。完了まで数分お待ちください。
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

import os, importlib.util
!pip install --upgrade -qqq uv

if importlib.util.find_spec("torch") is None or "COLAB_" in "".join(os.environ.keys()):
    try: import numpy, PIL; get_numpy = f"numpy=={numpy.__version__}"; get_pil = f"pillow=={PIL.__version__}"
    except: get_numpy = "numpy"; get_pil = "pillow"
    !uv pip install -qqq \
        "torch>=2.8.0" "triton>=3.4.0" {get_numpy} {get_pil} torchvision bitsandbytes "transformers==4.56.2" \
        "unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo" \
        "unsloth[base] @ git+https://github.com/unslothai/unsloth" \
        git+https://github.com/triton-lang/triton.git@0add68262ab0a2e33b84524346cb27cbb2787356#subdirectory=python/triton_kernels
elif importlib.util.find_spec("unsloth") is None:
    !uv pip install -qqq unsloth
!uv pip install --upgrade --no-deps transformers==4.56.2 tokenizers trl==0.22.2 unsloth unsloth_zoo

In [ ]:
# インストール完了の確認
print("✅ ライブラリのインストールが完了しました！")

---

## 📍 ステップ 2/7 - モデル読み込み

| ステップ | 状態 |
|---------|------|
| 1. 環境セットアップ | ✅ 完了 |
| **2. モデル読み込み** | 🔄 **いまここ** |
| 3. ChatGPTデータ取得 | ⏳ まだ |
| 4. データ準備 | ⏳ まだ |
| 5. 学習実行 | ⏳ まだ |
| 6. テスト | ⏳ まだ |
| 7. 保存 | ⏳ まだ |

GPT-OSS 20Bモデルを読み込みます。  
**4bit量子化を使うので、無料のT4 GPUでも動きます！**

In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 🤖 GPT-OSS 20B モデルの読み込み
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

from unsloth import FastLanguageModel
import torch

# 📊 設定パラメータ
max_seq_length = 1024  # 一度に処理できるトークン数（長い会話は分割されます）
dtype = None           # 自動検出（変更不要）

# モデルの読み込み
print("🔄 モデルを読み込み中...（数分かかります）")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gpt-oss-20b",  # 使用するモデル
    dtype = dtype,                        # 自動検出
    max_seq_length = max_seq_length,      # コンテキスト長
    load_in_4bit = True,                  # 4bit量子化でメモリ節約
    full_finetuning = False,              # LoRAを使用（メモリ効率が良い）
)

print("✅ モデルの読み込みが完了しました！")

### LoRAアダプターの設定

LoRA（Low-Rank Adaptation）を使うことで、モデル全体の **わずか0.02%** のパラメータだけを学習します。  
これにより、少ないメモリと短い時間で学習できます。

In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 🧩 LoRAアダプターの設定
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

model = FastLanguageModel.get_peft_model(
    model,
    r = 8,  # LoRAのランク（8, 16, 32, 64, 128 から選択。大きいほど表現力が向上しますが、GPUメモリも消費するようになります。）
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,  # 0が最適化されている
    bias = "none",     # "none"が最適化されている
    use_gradient_checkpointing = "unsloth",  # メモリ節約
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

print("✅ LoRAアダプターの設定が完了しました！")

---

## 📍 ステップ 3/7 - ChatGPTデータ取得

| ステップ | 状態 |
|---------|------|
| 1. 環境セットアップ | ✅ 完了 |
| 2. モデル読み込み | ✅ 完了 |
| **3. ChatGPTデータ取得** | 🔄 **いまここ** |
| 4. データ準備 | ⏳ まだ |
| 5. 学習実行 | ⏳ まだ |
| 6. テスト | ⏳ まだ |
| 7. 保存 | ⏳ まだ |

### 📥 ChatGPTの会話データを取得する方法

#### ステップ1: ChatGPTの設定を開く
1. [ChatGPT](https://chat.openai.com) にログイン
2. 左下の **自分の名前** をクリック
3. 「**Settings**」を選択

#### ステップ2: データをエクスポート
1. 「**Data controls**」タブを選択
2. 「**Export data**」をクリック
3. メールでダウンロードリンクが届きます（数分〜数時間）

#### ステップ3: ファイルを確認
- ダウンロードしたZIPを解凍
- `conversations.json` が学習データになります

---

> 💡 **まだエクスポートできていない場合**  
> 下のセルで「サンプルデータを使う」を選択すると、デモ用データで学習を試せます。

In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📂 データソース確認
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# 最初のセル(cell-4)で設定した内容を確認
print(f"📂 選択されたデータソース: {OPTIONS[data_source]}")

if use_chatgpt_export:
    print("\n📤 ChatGPTエクスポートを使用します")
    # アップロードがまだの場合
    import os
    if not os.path.exists("conversations.json"):
        print("⚠️ conversations.json がまだアップロードされていません")
        print("   最初のセルで「upload_now」にチェックを入れて再実行してください")
    else:
        print("✅ conversations.json が見つかりました")
else:
    print("\n📚 サンプルデータを使用します")

In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📤 追加アップロード（必要な場合のみ）
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

import os

if use_chatgpt_export and not os.path.exists("conversations.json"):
    print("📁 conversations.json をアップロードしてください")
    from google.colab import files
    uploaded = files.upload()
    
    if 'conversations.json' in uploaded:
        print("✅ アップロード完了！")
    else:
        print("⚠️ ファイルが見つかりません。サンプルデータに切り替えます")
        use_chatgpt_export = False
else:
    print("✅ データの準備ができています。次のセルに進んでください。")

---

## 📍 ステップ 4/7 - データ準備

| ステップ | 状態 |
|---------|------|
| 1. 環境セットアップ | ✅ 完了 |
| 2. モデル読み込み | ✅ 完了 |
| 3. ChatGPTデータ取得 | ✅ 完了 |
| **4. データ準備** | 🔄 **いまここ** |
| 5. 学習実行 | ⏳ まだ |
| 6. テスト | ⏳ まだ |
| 7. 保存 | ⏳ まだ |

アップロードしたデータを学習用の形式に変換します。

In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 🔄 ChatGPTエクスポートの変換関数
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

import json

def load_chatgpt_conversations(file_path="conversations.json"):
    """
    ChatGPTエクスポートファイルを学習用データに変換

    引数:
        file_path: conversations.jsonのパス

    戻り値:
        学習用の会話リスト
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    training_data = []

    for conversation in data:
        messages = []
        mapping = conversation.get('mapping', {})

        # メッセージを順番に取得
        for node_id, node in mapping.items():
            msg = node.get('message')
            if msg and msg.get('content', {}).get('parts'):
                role = msg['author']['role']
                parts = msg['content']['parts']
                if parts and isinstance(parts[0], str):
                    content = parts[0].strip()

                    # userとassistantのメッセージのみ抽出
                    if role in ['user', 'assistant'] and content:
                        messages.append({
                            "role": role,
                            "content": content
                        })

        # 有効な会話のみ追加（最低2ターン）
        if len(messages) >= 2:
            training_data.append({"messages": messages})

    print(f"✅ {len(training_data)} 件の会話を読み込みました")
    return training_data


def filter_conversations(data, min_turns=2, max_turns=20, keywords=None):
    """
    会話をフィルタリング

    - min_turns: 最小ターン数
    - max_turns: 最大ターン数（長すぎる会話を除外）
    - keywords: 含めたいキーワード（オプション）
    """
    filtered = []
    for conv in data:
        turns = len(conv['messages'])
        if min_turns <= turns <= max_turns:
            if keywords is None:
                filtered.append(conv)
            else:
                text = ' '.join([m['content'] for m in conv['messages']])
                if any(kw in text for kw in keywords):
                    filtered.append(conv)

    print(f"✅ フィルタリング後: {len(filtered)} 件")
    return filtered

print("✅ 変換関数を定義しました")

In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📊 データの読み込みと準備
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

from datasets import Dataset, load_dataset

def formatting_prompts_func(examples):
    """会話をGPT-OSS形式のテキストに変換"""
    convos = examples["messages"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return {"text": texts}

if use_chatgpt_export:
    # ChatGPTエクスポートを使用
    print("🔄 ChatGPTエクスポートを変換中...")
    raw_data = load_chatgpt_conversations("conversations.json")
    
    # フィルタリング（オプション）
    filtered_data = filter_conversations(raw_data, min_turns=2, max_turns=20)
    
    # Datasetに変換
    dataset = Dataset.from_list(filtered_data)
else:
    # サンプルデータを使用
    print("🔄 サンプルデータを読み込み中...")
    dataset = load_dataset("HuggingFaceH4/Multilingual-Thinking", split="train")
    
    # ShareGPT形式に標準化
    from unsloth.chat_templates import standardize_sharegpt
    dataset = standardize_sharegpt(dataset)

print(f"\n📊 データセット情報:")
print(f"   - 会話数: {len(dataset)} 件")
print(dataset)

In [ ]:
# テキスト形式に変換
dataset = dataset.map(formatting_prompts_func, batched=True)
print("✅ データの変換が完了しました！")

In [ ]:
# @title 👀 データのサンプルを確認（クリックで展開）
# @markdown 変換されたデータがどのような形式か確認できます

show_sample = True  # @param {type:"boolean"}

if show_sample and len(dataset) > 0:
    print("📝 1件目のデータ（一部）:")
    print("=" * 60)
    sample_text = dataset[0]['text']
    # 長すぎる場合は切り詰め
    if len(sample_text) > 1500:
        print(sample_text[:1500] + "\n\n... (以下省略)")
    else:
        print(sample_text)
    print("=" * 60)

---

## 📍 ステップ 5/7 - 学習実行

| ステップ | 状態 |
|---------|------|
| 1. 環境セットアップ | ✅ 完了 |
| 2. モデル読み込み | ✅ 完了 |
| 3. ChatGPTデータ取得 | ✅ 完了 |
| 4. データ準備 | ✅ 完了 |
| **5. 学習実行** | 🔄 **いまここ** |
| 6. テスト | ⏳ まだ |
| 7. 保存 | ⏳ まだ |

いよいよ学習を開始します！

In [ ]:
# @title ⚙️ 学習パラメータの設定
# @markdown 必要に応じて調整してください

# 📊 学習設定の調整ガイド
#
# max_steps: 学習ステップ数
#   - 30（テスト用・約10分）
#   - 100（軽い学習・約30分）
#   - 500（しっかり学習・約2時間）
#
# learning_rate: 学習率
#   - 2e-4（推奨：安定した学習）
#   - 1e-4（より慎重な学習）

max_steps = 30  # @param {type:"slider", min:10, max:500, step:10}
learning_rate = 2e-4  # @param {type:"number"}
batch_size = 1  # @param {type:"slider", min:1, max:4, step:1}

print(f"📊 学習設定:")
print(f"   - 学習ステップ数: {max_steps}")
print(f"   - 学習率: {learning_rate}")
print(f"   - バッチサイズ: {batch_size}")

In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 🎓 トレーナーの設定
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

from trl import SFTConfig, SFTTrainer

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    args = SFTConfig(
        per_device_train_batch_size = batch_size,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = max_steps,
        learning_rate = learning_rate,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

print("✅ トレーナーの設定が完了しました！")

In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 🎯 回答部分のみを学習対象にする
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# ユーザーの入力は学習せず、アシスタントの回答のみを学習します
# これにより、より精度の高い学習ができます

from unsloth.chat_templates import train_on_responses_only

gpt_oss_kwargs = dict(
    instruction_part = "<|start|>user<|message|>",
    response_part = "<|start|>assistant<|channel|>final<|message|>"
)

trainer = train_on_responses_only(
    trainer,
    **gpt_oss_kwargs,
)

print("✅ 回答部分のみを学習対象に設定しました！")

In [ ]:
# @title 💾 現在のメモリ使用状況
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"🖥️ GPU: {gpu_stats.name}")
print(f"💾 最大メモリ: {max_memory} GB")
print(f"📊 現在使用中: {start_gpu_memory} GB ({round(start_gpu_memory/max_memory*100, 1)}%)")

In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 🚀 学習開始！
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("🚀 学習を開始します...")
print(f"   設定: {max_steps} ステップ")
print("   ━" * 30)

trainer_stats = trainer.train()

print("\n" + "━" * 60)
print("🎉 学習が完了しました！")

In [ ]:
# @title 📊 学習結果のサマリー
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
training_time = trainer_stats.metrics['train_runtime']

print("📊 学習結果サマリー")
print("=" * 40)
print(f"⏱️ 学習時間: {round(training_time, 1)} 秒 ({round(training_time/60, 1)} 分)")
print(f"💾 使用メモリ: {used_memory} GB")
print(f"📈 学習用追加メモリ: {used_memory_for_lora} GB")
print("=" * 40)

---

## 📍 ステップ 6/7 - テスト

| ステップ | 状態 |
|---------|------|
| 1. 環境セットアップ | ✅ 完了 |
| 2. モデル読み込み | ✅ 完了 |
| 3. ChatGPTデータ取得 | ✅ 完了 |
| 4. データ準備 | ✅ 完了 |
| 5. 学習実行 | ✅ 完了 |
| **6. テスト** | 🔄 **いまここ** |
| 7. 保存 | ⏳ まだ |

学習したモデルをテストしてみましょう！

In [ ]:
# @title 💬 モデルにメッセージを送る
# @markdown 好きなメッセージを入力して、モデルの回答を確認できます

user_message = "Hello! Please introduce yourself."  # @param {type:"string"}
reasoning_effort = "medium"  # @param ["low", "medium", "high"]

# 日本語のデフォルトメッセージに変換（空の場合）
if user_message == "Hello! Please introduce yourself.":
    user_message = "こんにちは！自己紹介をしてください。"

messages = [
    {"role": "user", "content": user_message},
]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
    reasoning_effort = reasoning_effort,
).to("cuda")

# 推論レベルの説明
REASONING_DESC = {"low": "低（高速）", "medium": "中（バランス）", "high": "高（詳細）"}

print(f"💬 あなた: {user_message}")
print(f"🧠 推論レベル: {REASONING_DESC[reasoning_effort]}")
print("\n🤖 モデルの回答:")
print("-" * 40)

from transformers import TextStreamer
_ = model.generate(**inputs, max_new_tokens=256, streamer=TextStreamer(tokenizer))

### 💡 推論レベル（Reasoning Effort）について

GPT-OSSには3つの推論レベルがあります：

| レベル | 特徴 | 適した用途 |
|--------|------|------------|
| **low** | 高速、シンプルな回答 | 簡単な質問、チャット |
| **medium** | バランス型 | 一般的な質問 |
| **high** | 深い思考、詳細な回答 | 複雑な問題、分析 |

---

## 📍 ステップ 7/7 - 保存

| ステップ | 状態 |
|---------|------|
| 1. 環境セットアップ | ✅ 完了 |
| 2. モデル読み込み | ✅ 完了 |
| 3. ChatGPTデータ取得 | ✅ 完了 |
| 4. データ準備 | ✅ 完了 |
| 5. 学習実行 | ✅ 完了 |
| 6. テスト | ✅ 完了 |
| **7. 保存** | 🔄 **いまここ** |

学習したモデルを保存しましょう！

In [ ]:
# @title 💾 保存方法を選択
# @markdown どこに保存しますか？

save_location = "colab"  # @param ["colab", "gdrive", "huggingface"]

# 選択肢の説明（日本語）
SAVE_OPTIONS = {
    "colab": "Colabのファイル（一時的）",
    "gdrive": "Google Drive（永続保存）",
    "huggingface": "Hugging Face Hub（公開/共有）"
}

print(f"📁 保存先: {SAVE_OPTIONS[save_location]}")

In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 💾 モデルの保存
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

model_name = "my_gpt_oss_20b"

if save_location == "colab":
    # Colabの一時ファイルに保存（セッション終了で消えます）
    model.save_pretrained(model_name)
    print(f"✅ モデルを '{model_name}' に保存しました")
    print("⚠️ 注意: Colabのセッションが終了すると消えます")

elif save_location == "gdrive":
    # Google Driveに保存（永続的）
    from google.colab import drive
    print("🔗 Google Driveに接続中...")
    drive.mount('/content/drive')
    
    save_path = f"/content/drive/MyDrive/{model_name}"
    model.save_pretrained(save_path)
    print(f"✅ モデルを Google Drive に保存しました")
    print(f"   パス: {save_path}")

elif save_location == "huggingface":
    # Hugging Face Hubにアップロード
    print("📤 Hugging Face Hubにアップロードするには:")
    print("   1. https://huggingface.co/settings/tokens でトークンを取得")
    print("   2. 下のコードのコメントを解除して実行")
    print("")
    print('# model.push_to_hub("あなたのユーザー名/my_gpt_oss_20b", token="hf_...")')

---

## 🎉 完了！

おめでとうございます！あなた専用のGPT-OSS 20Bモデルが完成しました。

### 🔄 保存したモデルを読み込む方法

次回、保存したモデルを使いたい場合は以下のコードを使ってください：

In [ ]:
# 保存したモデルを読み込む例（必要な時にコメント解除）

if False:  # True に変更して実行
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "my_gpt_oss_20b",  # 保存時の名前
        max_seq_length = 1024,
        dtype = None,
        load_in_4bit = True,
    )
    print("✅ モデルを読み込みました！")

---

## 🆘 困ったときは

### よくあるエラーと解決法

<details>
<summary>❌ <b>CUDA out of memory</b></summary>

**原因**: GPUのメモリが足りません

**解決法**:
1. 「ランタイム」→「ランタイムを再起動」
2. `max_seq_length` を 512 に下げる
3. `batch_size` を 1 にする
</details>

<details>
<summary>❌ <b>No GPU runtime</b></summary>

**原因**: GPUが有効になっていません

**解決法**:
「ランタイム」→「ランタイムのタイプを変更」→「T4 GPU」を選択
</details>

<details>
<summary>❌ <b>conversations.json が読み込めない</b></summary>

**原因**: ファイル形式かアップロードの問題

**解決法**:
1. ファイルが正しくアップロードされているか確認
2. ZIPを解凍してから .json ファイルをアップロード
3. ファイル名が `conversations.json` か確認
</details>

<details>
<summary>❌ <b>学習が遅い・固まる</b></summary>

**原因**: データが多すぎる、またはGPUの問題

**解決法**:
1. `max_steps` を減らす（30程度でテスト）
2. データをフィルタリングして件数を減らす
3. ランタイムを再起動して再実行
</details>

---

### 💬 サポート

- Unsloth Discord: https://discord.gg/unsloth
- Unsloth ドキュメント: https://docs.unsloth.ai/
- GitHub: https://github.com/unslothai/unsloth

---

## 🚀 次のステップ

学習を終えたら、以下のような発展的な使い方もできます：

### 📚 もっと学ぶ

1. **強化学習（GRPO）でさらに賢く** - [Llama GRPO notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. **Ollamaで簡単に使う** - [Ollama保存 notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. **画像も理解するモデル** - [Vision Fine-tuning](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)

### 🔗 リンク

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>
</div>

---

このノートブックは [LGPL-3.0](https://github.com/unslothai/notebooks?tab=LGPL-3.0-1-ov-file#readme) ライセンスです。